# Imports and definitions

In [ ]:
import numpy as np
import qutip as qtp
import math
import cmath
import matplotlib.pyplot as plt
from qutip import *
from tqdm.notebook import tqdm
from scipy import interpolate
from scipy.integrate import solve_ivp
from scipy.integrate import quad
%matplotlib inline
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
blues = mpl.colormaps['Blues']
reds = mpl.colormaps['Reds']
import scqubits as scq
from scipy import constants
import pandas as pd
from matplotlib.legend_handler import HandlerTuple

In [ ]:
from matplotlib.colors import PowerNorm, ListedColormap
# Choose colormap
cmap_blue = plt.cm.Blues
cmap_red = plt.cm.Reds
cmap_green = plt.cm.Greens
cmap_purple = plt.cm.Purples
cmap_orange = plt.cm.Oranges

# Get the colormap colors
cmap_blue = cmap_blue(np.arange(plt.cm.Blues.N))
cmap_red = cmap_red(np.arange(plt.cm.Reds.N))
cmap_green = cmap_green(np.arange(plt.cm.Greens.N))
cmap_purple = cmap_purple(np.arange(plt.cm.Purples.N))
cmap_orange = cmap_orange(np.arange(plt.cm.Oranges.N))

# Set alpha
cmap_blue[:,-1] = np.linspace(0, 1, plt.cm.Blues.N)
cmap_red[:,-1] = np.linspace(0, 1, plt.cm.Reds.N)
cmap_green[:,-1] = np.linspace(0, 1, plt.cm.Greens.N)
cmap_purple[:,-1] = np.linspace(0, 1, plt.cm.Purples.N)
cmap_orange[:,-1] = np.linspace(0, 1, plt.cm.Oranges.N)

# Create new colormap
cmap_blue = ListedColormap(cmap_blue)
cmap_red = ListedColormap(cmap_red)
cmap_green = ListedColormap(cmap_green)
cmap_purple = ListedColormap(cmap_purple)
cmap_orange = ListedColormap(cmap_orange)

## Import data

In [ ]:
df = pd.read_csv(r'processed_data_fx8.csv')

#Fitted energy parameters - E_osc, g, Ej, Ec, El
energy_params = df['energy_params'].to_numpy()
energy_params = energy_params[~np.isnan(energy_params)]

df1 = pd.read_csv(r'res_fit_results.csv')

#phi_exts
phi_ext_disshift = df1['phi_ext_disshift'].to_numpy()
phi_ext_disshift = phi_ext_disshift[~np.isnan(phi_ext_disshift)]

#resonator params (GHz)
#frequency with qubit in |0>
frs_0_all = df1['fr_q0'].to_numpy()
frs_0_all = frs_0_all[~np.isnan(frs_0_all)]
#frequency with qubit in |1>
frs_1_all = df1['fr_q1'].to_numpy()
frs_1_all = frs_1_all[~np.isnan(frs_1_all)]
#resonator linewidth
kappa_exp = df1['kappa_avg'].to_numpy()
kappa_exp = kappa_exp[~np.isnan(kappa_exp)]

#chi (MHz)
chi_exp = df1['chi'].to_numpy()
chi_exp = chi_exp[~np.isnan(chi_exp)]

#resonator frequency with qubit in |0> and flux pulse amplitude
fp_amp = df1['flux_pulse_amp'].to_numpy()
fp_amp = fp_amp[~np.isnan(fp_amp)]

fr_q0_fp = df1['fr_q0_flux_pulse'].to_numpy()
fr_q0_fp = fr_q0_fp[~np.isnan(fr_q0_fp)]

In [ ]:
df = pd.read_csv(r'RO.csv')

#RO at sweet spot
#assignment error
raw_error_ss = df['RawErrorSS'].to_numpy()
raw_error_ss = raw_error_ss[~np.isnan(raw_error_ss)]
#SNR-limited error
SNR_error_ss = df['DiscrErrorSS'].to_numpy()
SNR_error_ss = SNR_error_ss[~np.isnan(SNR_error_ss)]
#integration time
t_ss = df['TimesSS'].to_numpy()
t_ss = t_ss[~np.isnan(t_ss)]
#readout frequency
ROfreq_ss = df['ROFreqSS'].to_numpy()
ROfreq_ss = ROfreq_ss[~np.isnan(ROfreq_ss)]

#RO with FPA scheme
#assignment error
raw_error_fpa = df['RawErrorFPA'].to_numpy()
raw_error_fpa = raw_error_fpa[~np.isnan(raw_error_fpa)]
#SNR-limited error
SNR_error_fpa = df['DiscrErrorFPA'].to_numpy()
SNR_error_fpa = SNR_error_fpa[~np.isnan(SNR_error_fpa)]
#integration time
t_fpa = df['TimesFPA'].to_numpy()
t_fpa = t_fpa[~np.isnan(t_fpa)]
#readout frequency
ROfreq_fpa = df['ROFreqFPA'].to_numpy()
ROfreq_fpa = ROfreq_fpa[~np.isnan(ROfreq_fpa)]

# Look at $\chi$, $\omega_{r}$, and $\Phi_{ext}$ in range of interest

## Dispersive shift at $\Phi_{ext}/\Phi_{0}$ = 0.6567
uses linear interpolation between nearest neighboring data points

In [ ]:
x1 = phi_ext_disshift[66]
x2 = phi_ext_disshift[67]
y1 = chi_exp[66]
y2 = chi_exp[67]

m = (y2-y1)/(x2-x1)
b = y1 - (m*x1)

ro_chi = m*0.6567 + b
ro_chi

In [ ]:
phi_trunc = np.array(phi_ext_disshift[:67])
chi_trunc = np.array(chi_exp[:67])
phi_trunc = np.append(phi_trunc, 0.6567)
chi_trunc = np.append(chi_trunc, ro_chi)

## Defines time points during ramp

In [ ]:
slope = 0.050/(phi_trunc[-1] - phi_trunc[0])
b = -1*(slope*phi_trunc[0])

t_ramping = []
for i in phi_trunc:
    t_ramping.append(i*slope + b)

## Resonator frequency
uses linear interpolation between nearest neighboring data points

In [ ]:
frs_0_MHz = np.array(frs_0_all)*10**3
frs_1_MHz = np.array(frs_1_all)*10**3

In [ ]:
x1 = phi_ext_disshift[66]
x2 = phi_ext_disshift[67]
y1 = frs_0_MHz[66]
y2 = frs_0_MHz[67]

m = (y2-y1)/(x2-x1)
b = y1 - (m*x1)

fr0_ro = m*0.6567 + b
fr0_ro #resonator frequency at readout point with qubit in |0>, not readout frequency

In [ ]:
x1 = phi_ext_disshift[66]
x2 = phi_ext_disshift[67]
y1 = frs_1_MHz[66]
y2 = frs_1_MHz[67]

m = (y2-y1)/(x2-x1)
b = y1 - (m*x1)

fr1_ro = m*0.6567 + b
fr1_ro #resonator frequency at readout point with qubit in |1>, not readout frequency

## Range of values during flux pulse ramp

In [ ]:
frs_0_MHz_trunc = np.array(frs_0_MHz[:67])
frs_1_MHz_trunc = np.array(frs_1_MHz[:67])
frs_0_MHz_trunc = np.append(frs_0_MHz_trunc, fr0_ro)
frs_1_MHz_trunc = np.append(frs_1_MHz_trunc, fr1_ro)

In [ ]:
fr_drive_MHz = ROfreq_fpa[0]*10**(-6) #readout frequenecy used in experiment

In [ ]:
#frequency between resonator with qubit in |0> and resonator with qubit in |1>
mid_freq_MHz = []
for i in range(len(frs_0_MHz_trunc)):
    mid_freq_MHz.append((frs_0_MHz_trunc[i]+frs_1_MHz_trunc[i])/2)

## Define global variables

In [ ]:
kappa = 2 * np.pi * kappa_exp[0] #resonator linewidith, 2*pi*MHz

In [ ]:
chi_SS = 2 * np.pi * chi_trunc[0] #2*pi*MHz
chi_RO = 2 * np.pi * chi_trunc[-1] #2*pi*MHz, chosen to be at Phi_ext/Phi_0 = 0.6567 based on measured data

In [ ]:
omega_drive = 2 * np.pi * fr_drive_MHz #2*pi*MHz
omega_0ss = 2 * np.pi * frs_0_MHz_trunc
omega_1ss = 2 * np.pi * frs_1_MHz_trunc

In [ ]:
Delta0 = []
Delta1 = []
for i in range(len(mid_freq_MHz)):
    d0 = 2*np.pi*(mid_freq_MHz[i]-fr_drive_MHz) - 2*np.pi*chi_trunc[i]
    d1 = 2*np.pi*(mid_freq_MHz[i]-fr_drive_MHz) + 2*np.pi*chi_trunc[i]
    Delta0.append(d0)
    Delta1.append(d1)

# Use numerics to get SNR at sweet-spot

In [ ]:
n_readout = 75 #average cavity photon number
tau = np.linspace(0.001, 1, 101) #integration time
#epsilon
ep_arg = (1/(Delta0[0]**2 + (kappa**2/4))) + (1/(Delta1[0]**2 + (kappa**2/4)))
ep = np.sqrt((2*n_readout)/(ep_arg))

In [ ]:
#solve differential equation to get alpha for two eigenstates (|0>, |1>)
def alphadot_0(alpha, time):
    dalpha0dt = 1j * chi_SS * alpha - (1/2) * kappa * alpha + ep
    return dalpha0dt

def alphadot_1(alpha, time):
    dalpha1dt = -1j * chi_SS * alpha - (1/2) * kappa * alpha + ep
    return dalpha1dt

alpha_init = [0+0j]

sol_alpha0 = solve_ivp(lambda time, alpha: alphadot_0(alpha, time), [tau[0], tau[-1]], alpha_init, t_eval=tau)
sol_alpha1 = solve_ivp(lambda time, alpha: alphadot_1(alpha, time), [tau[0], tau[-1]], alpha_init, t_eval=tau)

alpha0solution = sol_alpha0.y[0]
alpha1solution = sol_alpha1.y[0]

In [ ]:
alpha_out_0 = []
alpha_out_1 = []

#calculation of alpha_out from alpha and alpha_in
for i, a in enumerate(alpha0solution):
    aout0 = (ep / math.sqrt(kappa)) + math.sqrt(kappa) * a
    alpha_out_0.append(aout0)

for j, b in enumerate(alpha1solution):
    aout1 = (ep / math.sqrt(kappa)) + math.sqrt(kappa) * b
    alpha_out_1.append(aout1)
    
aout0_func = interpolate.interp1d(tau, alpha_out_0, fill_value="extrapolate")
aout1_func = interpolate.interp1d(tau, alpha_out_1, fill_value="extrapolate")

rawSNR_ss = []

#numerical integration to get SNR
for t in tau:
    M = []
    one = []
    zero = []
    tpts = np.linspace(0,t,1001)
    for i in tpts:
        alpha_zero = aout0_func(i)
        alpha_one = aout1_func(i)
        M.append(alpha_zero-alpha_one)
    SNRnumerator = np.sqrt(kappa)*abs(np.trapz(M))*np.diff(tpts)[0]
    SNRdenominator = math.sqrt(kappa * t)
    SNR = SNRnumerator / SNRdenominator
    rawSNR_ss.append(SNR)

## Include $\eta = 6.04\%$

In [ ]:
error_ss = []
error_loweff_ss = []
eta = 0.0604

for i in rawSNR_ss:
    error = (math.erfc(i/2)/2)
    error_ss.append(error)
    snr_low_eta = np.sqrt(eta)*i
    error_low_eta = (math.erfc(snr_low_eta/2)/2)
    error_loweff_ss.append(error_low_eta)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax

tick_fs = 9
label_fs = 10
lw = 1.5

###
raw_static, = ax1.plot(t_ss, raw_error_ss, color=cmap_red(0.7), linestyle='', marker='.', markersize=6, label=r'SS, Assign.(raw)')

# numerics
ax1.plot(np.array(tau)*10**3+40, error_loweff_ss, color=cmap_red(0.5), linestyle='--')

# data
snr_lim_static, = ax1.plot(t_ss, SNR_error_ss, color=cmap_red(0.9), linestyle='', marker='+', markersize=6, label=r'SS, SNR-limited')
ax1.tick_params(axis='y', pad=3, labelsize=tick_fs)
ax1.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax1.set_yscale('log')
ax1.set_xlim(100, 360)
ax1.set_ylim(0.009, 1)
ax1.set_xlabel('Integration Time (ns)', fontsize=label_fs)
ax1.set_ylabel('Readout Error', fontsize=label_fs, labelpad=2)
ax1.legend(handletextpad=0.25, frameon=False, handlelength = 1.0, fontsize = tick_fs, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(-0.03, -0.05))

# Use numerics to get SNR with FPA readout

## Solve Langevin equation analytically during 50 ns flux pulse ramping to get $\alpha$

In [ ]:
Delta0_func_t = interpolate.interp1d(t_ramping, Delta0, fill_value="extrapolate")
Delta1_func_t = interpolate.interp1d(t_ramping, Delta1, fill_value="extrapolate")

In [ ]:
#solve differential equation to get alpha for two eigenstates (|0>, |1>)
def alphadot_0(alpha, time):
    dalpha0dt = 1j * Delta0_func_t(time) * alpha - (1/2) * kappa * alpha + ep
    return dalpha0dt

def alphadot_1(alpha, time):
    dalpha1dt = -1j * Delta1_func_t(time) * alpha - (1/2) * kappa * alpha + ep
    return dalpha1dt

alpha_init = [0+0j]

sol_alpha0 = solve_ivp(lambda time, alpha: alphadot_0(alpha, time), [t_ramping[0], t_ramping[-1]], alpha_init, t_eval=t_ramping)
sol_alpha1 = solve_ivp(lambda time, alpha: alphadot_1(alpha, time), [t_ramping[0], t_ramping[-1]], alpha_init, t_eval=t_ramping)

alpha0solution = sol_alpha0.y[0]
alpha1solution = sol_alpha1.y[0]

## Transformation of $\alpha$ to $\alpha_{out}$ using interpolation during flux pulse ramp

In [ ]:
alpha_out_0 = []
alpha_out_1 = []

#calculation of alpha_out from alpha and alpha_in
for i, a in enumerate(alpha0solution):
    aout0 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * a
    alpha_out_0.append(aout0)

for j, b in enumerate(alpha1solution):
    aout1 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * b
    alpha_out_1.append(aout1)
    
aout0_func = interpolate.interp1d(t_ramping, alpha_out_0, fill_value="extrapolate")
aout1_func = interpolate.interp1d(t_ramping, alpha_out_1, fill_value="extrapolate")

rawSNR_ramp = []

#integration to get measurement operator which allows for calculation of SNR
for t in t_ramping:
    M_ramp = []
    tpts = np.linspace(0,t,1001)
    for i in tpts:
        alpha_zero = aout0_func(i)
        alpha_one = aout1_func(i)
        M_ramp.append(alpha_zero-alpha_one)
    SNRnumerator = np.sqrt(kappa)*abs(np.trapz(M_ramp))*np.diff(tpts)[0]
    SNRdenominator = math.sqrt(kappa * t)
    SNR_ramp = SNRnumerator / SNRdenominator
    rawSNR_ramp.append(SNR_ramp)

## Solve Langevin equation for $\alpha$ and calculate SNR after flux pulse has reached its maximum amplitude at $\Phi_{ext}/\Phi_{0}$ = 0.6567

In [ ]:
def alphadot_0_fp(alpha, time): #while flux pulse is on, keeping qubit at readout point
    dalpha0dt_fp = 1j * Delta0[-1] * alpha - (1/2) * kappa * alpha + ep
    return dalpha0dt_fp

def alphadot_1_fp(alpha, time): #after flux pulse ramp
    dalpha1dt_fp = -1j * Delta1[-1] * alpha - (1/2) * kappa * alpha + ep
    return dalpha1dt_fp

#initial point defined as last point during flux pulse ramp
alpha_init_0 = [alpha0solution[-1]]
alpha_init_1 = [alpha1solution[-1]]

t_fp = np.linspace(t_ramping[-1], 1, 1901) #once flux pulse has reached max amplitude, up to 1 us

#solve differential equation to get alpha
sol_alpha0_fp = solve_ivp(lambda time, alpha: alphadot_0_fp(alpha, time), [t_fp[0], t_fp[-1]], alpha_init_0, t_eval=t_fp)
sol_alpha1_fp = solve_ivp(lambda time, alpha: alphadot_1_fp(alpha, time), [t_fp[0], t_fp[-1]], alpha_init_1, t_eval=t_fp)

alpha0solution_fp = sol_alpha0_fp.y[0]
alpha1solution_fp = sol_alpha1_fp.y[0]

aout0 = []
aout1 = []

#calculation of alpha_out from alpha and alpha_in
for a in alpha0solution_fp:
    alphaout0 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * a
    aout0.append(alphaout0)

for b in alpha1solution_fp:
    alphaout1 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * b
    aout1.append(alphaout1)
    
aout0_func_fp = interpolate.interp1d(t_fp, aout0, fill_value="extrapolate")
aout1_func_fp = interpolate.interp1d(t_fp, aout1, fill_value="extrapolate")

rawSNR_fp = []

#integration to get measurement operator which allows for calculation of SNR during flux pulse at max amplitude
for t in t_fp:
    M = []
    tps = np.linspace(0,t,1001)
    for tp in tps:
        alpha_zero = aout0_func_fp(tp) if tp>0.05 else aout0_func(tp)
        alpha_one = aout1_func_fp(tp) if tp>0.05 else aout1_func(tp)
        M.append(alpha_zero-alpha_one)
    SNRnum = np.sqrt(kappa) * abs(np.trapz(M)*np.diff(tps)[0])
    SNRdenom = math.sqrt(kappa * t)
    SNR = SNRnum / SNRdenom
    rawSNR_fp.append(SNR)

#concatenate time and SNR arrays to capture during and after flux pulse
totalTime = np.concatenate((t_ramping, t_fp))
totalRawSNR = np.concatenate((rawSNR_ramp, rawSNR_fp))

# Conversion to readout error and visualization

In [ ]:
eta = 0.0604
snr_loweff_fpa = np.sqrt(eta)*totalRawSNR

error_loweff_fpa = []

for i in snr_loweff_fpa:
    error = (math.erfc(i/2)/2)
    error_loweff_fpa.append(error)

error_loweff_ss = []
for i in rawSNR_ss:
    snr_low_eta = np.sqrt(eta)*i
    error_low_eta = (math.erfc(snr_low_eta/2)/2)
    error_loweff_ss.append(error_low_eta)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax

tick_fs = 9
label_fs = 10
lw = 1.5

# numerics, 40ns offset to account for difference in measurement acquisition times
ax1.plot(np.array(tau)*10**3+40, error_loweff_ss, color=cmap_orange(0.9), linestyle='--')
ax1.plot(np.array(totalTime)*10**3+40, error_loweff_fpa, color=cmap_purple(0.9), linestyle='--')

# data
snr_lim_static, = ax1.plot(t_ss, SNR_error_ss, color=cmap_red(0.9), linestyle='', marker='+', markersize=6, label=r'SS, SNR-limited')
snr_lim_fpa, = ax1.plot(t_fpa, SNR_error_fpa, color=cmap_blue(0.9), linestyle='', marker='+', markersize=6, label=r'FPA, SNR-limited')
ax1.tick_params(axis='y', pad=3, labelsize=tick_fs)
ax1.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax1.set_yscale('log')
ax1.set_xlim(100, 360)
ax1.set_ylim(0.009, 1)
ax1.set_xlabel('Integration Time (ns)', fontsize=label_fs)
ax1.set_ylabel('Readout Error', fontsize=label_fs, labelpad=2)
ax1.legend(handletextpad=0.25, frameon=False, handlelength = 1.0, fontsize = tick_fs, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(-0.03, -0.05))

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250, sharex=True)

fig.subplots_adjust(bottom = 0.1, top=0.9, left=0.12, right=0.8, wspace=0.4, hspace=0.25)
fig.set_size_inches(3.40457, 2.25)
font = {'size' : 8}
mpl.rc('font', **font)

label_fs = 8
tick_fs = 8
legend_fs = 8

lw = 2

num_static, = ax.plot(np.array(tau)*10**3+40, error_loweff_ss, color=cmap_red(0.6), linestyle='--', lw=lw, label=r'SS, Numerics')
num_fpa, = ax.plot(np.array(totalTime)*10**3+40, error_loweff_fpa, color=cmap_blue(0.6), linestyle='--', lw=lw, label=r'FPA, Numerics')

raw_static, = ax.plot(t_ss, raw_error_ss, color=cmap_red(0.7), linestyle='', marker='.', markersize=6, label=r'SS, Assign.(raw)')
raw_fpa, = ax.plot(t_fpa, raw_error_fpa, color=cmap_blue(0.7), linestyle='', marker='.', markersize=6, label=r'FPA, Assign.(raw)')
snr_lim_static, = ax.plot(t_ss, SNR_error_ss, color=cmap_red(0.9), linestyle='', marker='+', markersize=6, label=r'SS, SNR-limited')
snr_lim_fpa, = ax.plot(t_fpa, SNR_error_fpa, color=cmap_blue(0.9), linestyle='', marker='+', markersize=6, label=r'FPA, SNR-limited')
ax.tick_params(axis='y', pad=2, labelsize=tick_fs)
ax.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax.set_yscale('log')
ax.set_xlim(100, 360)
ax.set_ylim(0.0005, 1)
ax.set_xlabel('Integration Time (ns)', fontsize=label_fs)
ax.set_ylabel('Readout Error', fontsize=label_fs, labelpad=2)
txt1 = ax.annotate(r'SS', xy=(4.5,36) , size=legend_fs, xycoords="axes points", color=cmap_red(0.9))
txt2 = ax.annotate(r'FPA', xy=(20,36) , size=legend_fs, xycoords="axes points", color=cmap_blue(0.9))
l = ax.legend([(num_static, num_fpa), (raw_static, raw_fpa), (snr_lim_static, snr_lim_fpa)], [r"Numerics", r"Assign.(raw)", r"SNR-limited"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=0.6, frameon=False, handlelength = 4, fontsize = tick_fs, labelspacing=0.5, 
               loc='lower left', bbox_to_anchor=(-0.02, -0.03))
plt.tight_layout()